In [ ]:
import pandas as pd
import time
from lxml import html
import requests
df = pd.read_csv(r"C:\Users\lyons\OneDrive\Desktop\IML-FALL2025---Voter-Bias\original_data\college_football_polls_original.csv")
dff = pd.read_csv(r'C:\Users\lyons\OneDrive\Desktop\IML-FALL2025---Voter-Bias\original_data\college_basketball_polls_original.csv')
#run everything after cooldown passes -> did it too many times while testing whoops 

In [ ]:
#inital test
season_2014 = df[df["Season"] == '2014']
long2014 = season_2014.melt(
    id_vars=['Season', 'Week', 'Pollster'],
    value_vars=[str(i) for i in range(1, 26)],
    var_name='Rank',
    value_name='Team'
).dropna(subset=['Team'])
rank_points = {str(i): 26 - i for i in range(1, 26)}
long2014['Points'] = long2014['Rank'].map(rank_points)
df2014 = long2014.groupby(['Season', 'Week', 'Team'], as_index=False)['Points'].sum()
df2014 = df2014.rename(columns={'Points': 'Total_Points'})
url = 'https://www.sports-reference.com/cfb/years/2014-standings.html'
table = pd.read_html(url)
team_df = table[0]
team_df = team_df.rename(columns={
    'Unnamed: 1_level_0': 'School',
    'Unnamed: 2_level_0': 'Conf'
})
team_df = team_df[['School', 'Conf']]
team_df.columns = team_df.columns.get_level_values(0)
team_df = team_df.dropna()
team_df = team_df[team_df.School != 'School']
team_df.School = team_df.School.str.lower().str.replace(" ", "-").str.replace("&", "and").str.replace("(", "").str.replace(")", "").str.replace(".", "").str.replace("'", "")   

school_to_mascot = df2014['Team'].unique().tolist()
schools = team_df.School.unique().tolist()
map_school_to_mascot = {}
for school in schools:
    matches = [t for t in school_to_mascot if t.startswith(school)]
    if matches:
        map_school_to_mascot[school] = matches[0]
    else: 
        continue

team_df['updated school'] = team_df['School'].map(map_school_to_mascot)
team_df = team_df.dropna()
conf_map_2014 = dict(zip(team_df['updated school'], team_df['Conf']))
df2014['Conference'] = df2014['Team'].map(conf_map_2014)
df2014 = df2014[['Season', 'Week', 'Team', 'Conference', 'Total_Points']]

In [ ]:
#both cfb and cbb applicable
def standardize_conference_name(conf: str) -> str:
    if pd.isna(conf):
        return None
    conf = (
        conf.lower()
        .replace(" ", "-")
        .replace("&", "and")
        .replace("(", "")
        .replace(")", "")
        .replace(".", "")
        .replace("'", "")
    )
    for suffix in ["-east", "-west", "-north", "-south"]:
        if conf.endswith(suffix):
            conf = conf.replace(suffix, "")
    return conf

In [ ]:
#cfb
def process_poll_data_cfb(df, year):
    season_df = df[df["Season"] == str(year)]
    long_df = season_df.melt(
        id_vars=['Season', 'Week', 'Pollster'],
        value_vars=[str(i) for i in range(1, 26)],
        var_name='Rank',
        value_name='Team'
    ).dropna(subset=['Team'])
    rank_points = {str(i): 26 - i for i in range(1, 26)}
    long_df['Points'] = long_df['Rank'].map(rank_points)
    poll_df = long_df.groupby(['Season', 'Week', 'Team'], as_index=False)['Points'].sum()
    poll_df = poll_df.rename(columns={'Points': 'Total_Points'})
    url = f'https://www.sports-reference.com/cfb/years/{year}-standings.html' # <- link is different for cbb
    table = pd.read_html(url)[0]
    team_df = table.rename(columns={'Unnamed: 1_level_0': 'School', 'Unnamed: 2_level_0': 'Conf'})
    team_df = team_df[['School', 'Conf']]
    team_df.columns = team_df.columns.get_level_values(0)
    team_df = team_df.dropna()
    team_df = team_df[team_df.School != 'School']
    team_df.School = (
        team_df.School.str.lower()
        .str.replace(" ", "-", regex=False)
        .str.replace("&", "and", regex=False)
        .str.replace("(", "", regex=False)
        .str.replace(")", "", regex=False)
        .str.replace(".", "", regex=False)
        .str.replace("'", "", regex=False)
    )

    team_df['Conf'] = team_df['Conf'].apply(standardize_conference_name)
    
    school_to_mascot = poll_df['Team'].unique().tolist()
    schools = team_df.School.unique().tolist()
    map_school_to_mascot = {}
    for school in schools:
        matches = [t for t in school_to_mascot if t.startswith(school)]
        if matches:
            map_school_to_mascot[school] = matches[0]
    team_df['updated_school'] = team_df['School'].map(map_school_to_mascot)
    team_df = team_df.dropna()
    conf_map = dict(zip(team_df['updated_school'], team_df['Conf']))
    poll_df['Conference'] = poll_df['Team'].map(conf_map)
    poll_df = poll_df[['Season', 'Week', 'Team', 'Conference', 'Total_Points']]
    return poll_df

In [ ]:
#cfb
def process_all_years_cfb(df):
    all_dfs = []
    for year in sorted(df.Season.unique()):
        try:
            season_df = process_poll_data_cfb(df, year)
            all_dfs.append(season_df)
            print(f"Processed {year}")
            time.sleep(5)#
        except Exception as e:
            print(f"Skipped {year} due to error: {e}")
    all_years_df = pd.concat(all_dfs, ignore_index = True)
    return all_years_df

In [ ]:
# ret1 = process_all_years_cfb(df)
# ret1.to_csv('cfb_conf_aff.csv', index = False)

In [ ]:
# #cbb
# def process_poll_data_cbb(df, year):
#     season_df = df[df["Season"] == str(year)]
#     long_df = season_df.melt(
#         id_vars=['Season', 'Week', 'Pollster'],
#         value_vars=[str(i) for i in range(1, 26)],
#         var_name='Rank',
#         value_name='Team'
#     ).dropna(subset=['Team'])
#     rank_points = {str(i): 26 - i for i in range(1, 26)}
#     long_df['Points'] = long_df['Rank'].map(rank_points)
#     poll_df = long_df.groupby(['Season', 'Week', 'Team'], as_index=False)['Points'].sum()
#     poll_df = poll_df.rename(columns={'Points': 'Total_Points'})
#     url = f'https://www.sports-reference.com/cbb/seasons/men/{year}.html' # <- link for college mens basketball
#     table = pd.read_html(url)[0]
#     team_df = table.rename(columns={'Unnamed: 1_level_0': 'School', 'Unnamed: 2_level_0': 'Conf'})
#     team_df = team_df[['School', 'Conf']]
#     team_df.columns = team_df.columns.get_level_values(0)
#     team_df = team_df.dropna()
#     team_df = team_df[team_df.School != 'School']
#     team_df.School = (
#         team_df.School.str.lower()
#         .str.replace(" ", "-", regex=False)
#         .str.replace("&", "and", regex=False)
#         .str.replace("(", "", regex=False)
#         .str.replace(")", "", regex=False)
#         .str.replace(".", "", regex=False)
#         .str.replace("'", "", regex=False)
#     )

#     team_df['Conf'] = team_df['Conf'].apply(standardize_conference_name)
    
#     school_to_mascot = poll_df['Team'].unique().tolist()
#     schools = team_df.School.unique().tolist()
#     map_school_to_mascot = {}
#     for school in schools:
#         matches = [t for t in school_to_mascot if t.startswith(school)]
#         if matches:
#             map_school_to_mascot[school] = matches[0]
#     team_df['updated_school'] = team_df['School'].map(map_school_to_mascot)
#     team_df = team_df.dropna()
#     conf_map = dict(zip(team_df['updated_school'], team_df['Conf']))
#     poll_df['Conference'] = poll_df['Team'].map(conf_map)
#     poll_df = poll_df[['Season', 'Week', 'Team', 'Conference', 'Total_Points']]
#     return poll_df

In [ ]:
# cbb
def process_poll_data_cbb(df, year):
    season_df = df[df["Season"] == str(year)]
    long_df = season_df.melt(
        id_vars=['Season', 'Week', 'Pollster'],
        value_vars=[str(i) for i in range(1, 26)],
        var_name='Rank',
        value_name='Team'
    ).dropna(subset=['Team'])

    rank_points = {str(i): 26 - i for i in range(1, 26)}
    long_df['Points'] = long_df['Rank'].map(rank_points)

    poll_df = long_df.groupby(['Season', 'Week', 'Team'], as_index=False)['Points'].sum()
    poll_df = poll_df.rename(columns={'Points': 'Total_Points'})

    # Basketball URL
    url = f'https://www.sports-reference.com/cbb/seasons/men/{year}.html'
    page = requests.get(url)
    tables = pd.read_html(page.text, attrs={"id": "coaches-polls"})

    if tables:
        team_df = tables[0]

        # Fix the weird Unnamed headers
        if isinstance(team_df.columns, pd.MultiIndex):
            team_df.columns = [
                " ".join([str(c) for c in col if "Unnamed" not in str(c)]).strip()
                for col in team_df.columns
            ]

        # Keep only School and Conf
        team_df = team_df[['School', 'Conf']].copy()
    else:
        raise ValueError("Could not find coaches-polls table")

    # Clean up School column
    # team_df = team_df.dropna()
    # team_df = team_df[team_df.School != 'School']
    # team_df.School = (
    #     team_df.School.str.lower()
    #     .str.replace(" ", "-", regex=False)
    #     .str.replace("&", "and", regex=False)
    #     .str.replace("(", "", regex=False)
    #     .str.replace(")", "", regex=False)
    #     .str.replace(".", "", regex=False)
    #     .str.replace("'", "", regex=False)
    # )

    # Standardize conferences
    # team_df['Conf'] = team_df['Conf'].apply(standardize_conference_name)

    # Match to poll_df teams
    school_to_mascot = poll_df['Team'].unique().tolist()
    schools = team_df.School.unique().tolist()
    map_school_to_mascot = {}
    for school in schools:
        matches = [t for t in school_to_mascot if t.startswith(school)]
        if matches:
            map_school_to_mascot[school] = matches[0]

    team_df['updated_school'] = team_df['School'].map(map_school_to_mascot)
    team_df = team_df.dropna()

    conf_map = dict(zip(team_df['updated_school'], team_df['Conf']))
    poll_df['Conference'] = poll_df['Team'].map(conf_map)

    poll_df = poll_df[['Season', 'Week', 'Team', 'Conference', 'Total_Points']]
    return poll_df


In [ ]:
url = "https://www.sports-reference.com/cbb/seasons/men/2014.html"
page = requests.get(url)

# Grab only the coaches-polls table by ID
tables = pd.read_html(page.text, attrs={"id": "coaches-polls"})
coaches_df = tables[0]

In [ ]:
coaches_df 
if isinstance(coaches_df.columns, pd.MultiIndex):
    coaches_df.columns = [" ".join([str(c) for c in col if "Unnamed" not in str(c)]).strip()
                       for col in coaches_df.columns]

# Keep just the School + Conf columns
coaches_df = coaches_df[['School', 'Conf']].copy()
coaches_df

In [37]:
process_poll_data_cbb(dff, 2014)

,Season,Week,Team,Conference,Total_Points


In [ ]:
#cbb
def process_all_years_cbb(df):
    all_dfs = []
    for year in sorted(df.Season.unique()):
        try:
            season_df = process_poll_data_cbb(df, year)
            all_dfs.append(season_df)
            print(f"Processed {year}")
            time.sleep(10)#
        except Exception as e:
            print(f"Skipped {year} due to error: {e}")
    all_years_df = pd.concat(all_dfs, ignore_index = True)
    return all_years_df

In [ ]:
# #cbb
# ret2 = process_all_years_cbb(dff)
# ret2.to_csv('conf_aff_cbb.csv', index = False)